<a href="https://colab.research.google.com/github/MoohShadox/Web_Semantique_Workshop/blob/master/Lab%2003%20-%20OwlReady.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install owlready2
from owlready2 import *
!git clone https://github.com/MoohShadox/Web_Semantique_Workshop

     |████████████████████████████████| 20.8MB 1.5MB/s 
  Created wheel for owlready2: filename=Owlready2-0.23-cp36-cp36m-linux_x86_64.whl size=20259998 sha256=1cb834d66cb519653bfcccbf2f9791ab7221a59f3ba96a2b4de705cd70431f8d
  Stored in directory: /root/.cache/pip/wheels/a4/f5/cd/357e802aa4c8313bf7ee78aa89b9791cef93661ff112259520
Successfully built owlready2
Cloning into 'Web_Semantique_Workshop'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 65 (delta 27), reused 45 (delta 15), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [25]:
!pip install rdflib
import rdflib


     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 


# Lab 02 : OWLReady 
Dans ce lab nous allons étudier le fonctionnement de OWLReady et les possibilités que cette librairie offre, nous aborderons notamment les questions suivantes : 
- Manipuler des ontologies déja existantes
- Comment enrichir une ontologie
- Comment créer et ajouter des individus
- Comment éditer des régles d'inférences 
- Comment exécuter le raisonneur et les différents raisonneurs qui existent (Hermit, Pellet etc..) 
- Comment voire ce que le raisonner a inféré et notamment comment vérifier qu'une classe est inconsistante.
- Comment transformer une ontologie en graphe rdflib afin de pouvoir exécuter des requêtes SPARQL dessus.

## 1- Récupérer et Manipuler des ontologies 
Avant toute chose il faut savoir que si les ontologies peuvent être sérialisées en un fichier .owl ce n'est pas la représentation la plus pratique pour stocker des grands volumes de données, ils serrons plûtot enregistrées dans des quadstore qui ont pour avantage d'être indexées.

Cette première ligne servira donc a fixer le back end ce qui signifie trois chose : 
- Si le fichier existe déja la ligne va automatiquement récupérer les ontologies qui sont dessus
- Si le fichier n'existe pas et que vous n'avez pas encore commencé la construction de l'ontologie, le fichier serra crée.
- Si le fichier n'existe pas mais que vous avez déja commencé a construire l'ontologie (si vous avez exécuté le notebook puis que vous êtes remontés pour executer cette cellule par exemple) ça renverra une erreur.

In [0]:
default_world.set_backend(filename = "file_back3.sqlite3", exclusive = False)

Ensuite, il est possible de charger une ontologie a partir d'un fichier OWL sérialisé en RDF/XML, ce fichier peut provenir de l'exercice précédent qui génére une ontologie avec RDFLib ou directement de protégé.
Dans cette application nous utiliserons une ontologie faite sur protégé.

In [3]:
onto = get_ontology("file:///content/Web_Semantique_Workshop/Amazone_Ontology.owl").load()
!cat /content/Web_Semantique_Workshop/Amazone_Ontology.owl

<?xml version="1.0"?>
<rdf:RDF xmlns="http://www.semanticweb.org/mohamed/ontologies/2020/3/Amazone_Ont#"
     xml:base="http://www.semanticweb.org/mohamed/ontologies/2020/3/Amazone_Ont"
     xmlns:owl="http://www.w3.org/2002/07/owl#"
     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
     xmlns:xml="http://www.w3.org/XML/1998/namespace"
     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
    <owl:Ontology rdf:about="http://www.semanticweb.org/mohamed/ontologies/2020/3/Amazone_Ont"/>
    


    <!-- 
    ///////////////////////////////////////////////////////////////////////////////////////
    //
    // Object Properties
    //
    ///////////////////////////////////////////////////////////////////////////////////////
     -->

    


    <!-- http://www.semanticweb.org/mohamed/ontologies/2020/3/Amazone_Ont#also_bought -->

    <owl:ObjectProperty rdf:about="http://www.semanticweb.org/mohamed/ontologies/2020/3/Amazone_On

Maintenant que nous avons récupéré notre ontologie voici quelques exemples de méthodes permettant de l'explorer : 

In [4]:
#Afficher les classes : 
for i in onto.classes():
    print(i)

Amazone_Ontology.Article
Amazone_Ontology.Category
Amazone_Ontology.Brands
Amazone_Ontology.Main_Category
Amazone_Ontology.Other_Categories


In [5]:
#Afficher les propriétés : 
for i in onto.object_properties():
    print(i)

Amazone_Ontology.also_bought
Amazone_Ontology.also_view
Amazone_Ontology.appartientCategory
Amazone_Ontology.brandOf
Amazone_Ontology.hasBrand
Amazone_Ontology.contientArticle


In [0]:
for i in onto.individuals():
    print(i)

Nous remarquons que l'ontologie ne contient pas d'individus mais nous allons y remédier dans la prochaine partie.

## 2- Créer des individus
Pour accéder aux classes présentes dans l'ontologie nous pouvons utiliser le générateur de classes qu'on transformera en liste pour l'occasion.

In [0]:
class_article = list(onto.classes())[0]
A = class_article()

In [8]:
for i in onto.individuals():
    print(i)

Amazone_Ontology.article1


In [0]:
L = onto.search(type = class_article)

Et cette fois ci on remarquera qu'on a bien un individu qui c'est crée, réexecutais plusieurs fois la première cellule vous pourrez constater que l'instruction crée bien un individu.

In [0]:
relation_alsobought = list(onto.object_properties())[0]
relation_alsobought.python_name = "also_bought"

In [11]:
relation_alsobought

Amazone_Ontology.also_bought

In [12]:
print(L)

[Amazone_Ontology.article1]


Cependant notre petite astuce ne permet pas de créer des relations, ainsi, on priviligiera une façon native qui est beaucoup plus pratique pour créer nos relations, cependant, manipuler une ontologie crées sur protégé peut s'avérer particulièrement utile afin d'en récupérer les individus et les concepts.

## 3- Enrichir une ontologie 
Cette fois ci, nous allons essayer de reconstruire une nouvelle ontologie de zéro en utilisant owlready. 
Ce qui est intéressant avec cette façon de faire c'est que nous allons créer des classes Python en même temps que les classes représentant les concepts correspondant dans l'ontologie.

In [0]:
#On crée une nouvelle ontologie 
onto = get_ontology("https://urlinvente/monontologie")

with onto:
    
    #On crée une classe Animal et une classe Plante qui héritent toutes deux de Thing
    #On peut même créer des méthodes dans les classes comme en orienté objet
    class Animal(Thing):
        def presente_toi(self):
            print("je suis un animal !")
        
    class Plante(Thing):
        pass
    
    #On précise que les deux classe sont disjointes
    AllDisjoint([Animal, Plante])
    
    #On crée une propriété qui va avoir comme domain Animal et comme range Thing
    class mange(Animal >> Thing):
        pass
    
    class nemangepas(Animal >> Animal):
        pass
    
    """On crée une dataproperty cette fois sans utiliser la notation abrégée ce qui nous permet d'hériter a la fois
    de DataProperty mais également de FunctionalProperty qui permet donc de préciser que chaque animal a un un nom
    unique
    """
    class nomOfficiel(DataProperty, FunctionalProperty):
        domain = [Animal]
        range = [str]
        
    """
    Maintenant on va créer une ou deux classes par equivalence
    """
    class carnivore(Thing):
        equivalent_to = [Animal & mange.some(Animal)]
        def presente_toi(self):
            print("je suis un carnivore !")
            
            
    class herbivore(Thing):
        equivalent_to = [Animal & mange.only(Plante)]
        
    class nonsense(Thing):
        equivalent_to = [Animal & Plante]


A présent on peut commencer a créer nos individus et ensuite nous pourrons les manipuler en leur ajoutant des relations.

In [0]:
Lion = Animal()
Arbre = Plante()
Tigre = carnivore()
Brebis = Animal()

In [0]:
Brebis.mange = [Arbre]
Lion.mange = [Brebis]
Lion.nomOfficiel = "Lion"
Tigre.nomOfficiel = "Tigre"
Brebis.nomOfficiel = "Brebis"

In [16]:
onto.save(file = "out.owl", format = "ntriples")
!cat out.owl

<https://urlinvente/monontologie> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Ontology> .
<https://urlinvente/monontologie#Animal> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<https://urlinvente/monontologie#Animal> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.w3.org/2002/07/owl#Thing> .
<https://urlinvente/monontologie#Plante> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<https://urlinvente/monontologie#Plante> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.w3.org/2002/07/owl#Thing> .
<https://urlinvente/monontologie#Animal> <http://www.w3.org/2002/07/owl#disjointWith> <https://urlinvente/monontologie#Plante> .
<https://urlinvente/monontologie#mange> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty> .
<https://urlinvente/monontologie#mange> <http://www.w3.org/2000/01/rdf-schema#dom

## Inférence et raisonnement 
A présent voyons comment nous pouvons ajouter des régles d'inférences afin d'inférer des nouvelles connaissances a partir des connaissances existantes 

In [0]:
with onto:
    rule = Imp()
    rule.set_as_rule("""carnivore(?a1), carnivore(?a2) -> nemangepas(?a1, ?a2)""")

In [18]:
"""
On execute les méthodes ici avant le raisonneur
"""
Lion.presente_toi()
Tigre.presente_toi()
Brebis.presente_toi()

je suis un animal !
je suis un carnivore !
je suis un animal !


In [19]:
with onto:
    sync_reasoner(infer_property_values = True)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.6/dist-packages/owlready2/hermit:/usr/local/lib/python3.6/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpcu8x8yfu -Y


* Owlready * Adding relation monontologie.carnivore1 nemangepas monontologie.carnivore1
* Owlready * Adding relation monontologie.carnivore1 nemangepas monontologie.animal1
* Owlready * Adding relation monontologie.animal1 nemangepas monontologie.carnivore1
* Owlready * Adding relation monontologie.animal1 nemangepas monontologie.animal1


* Owlready2 * HermiT took 1.792048454284668 seconds
* Owlready * Equivalenting: monontologie.nonsense owl.Nothing
* Owlready * Reparenting monontologie.herbivore: {owl.Thing} => {monontologie.Animal}
* Owlready * Reparenting monontologie.carnivore: {owl.Thing} => {monontologie.Animal}
* Owlready * Reparenting monontologie.animal1: {monontologie.Animal} => {monontologie.carnivore}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [20]:
"""
Voyons voire si les changement de classes affectent l'execution des méthodes
"""
Lion.presente_toi()
Tigre.presente_toi()
Brebis.presente_toi()

je suis un carnivore !
je suis un carnivore !
je suis un animal !


In [21]:
with onto:
    sync_reasoner_pellet(infer_property_values = True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.6/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.6/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python

In [22]:
onto.save(file = "out.owl", format = "ntriples")
!cat out.owl
default_world.save()

<https://urlinvente/monontologie> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Ontology> .
<https://urlinvente/monontologie#Animal> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<https://urlinvente/monontologie#Animal> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.w3.org/2002/07/owl#Thing> .
<https://urlinvente/monontologie#Plante> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<https://urlinvente/monontologie#Plante> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.w3.org/2002/07/owl#Thing> .
<https://urlinvente/monontologie#Animal> <http://www.w3.org/2002/07/owl#disjointWith> <https://urlinvente/monontologie#Plante> .
<https://urlinvente/monontologie#mange> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty> .
<https://urlinvente/monontologie#mange> <http://www.w3.org/2000/01/rdf-schema#dom

In [26]:
import rdflib
graph = default_world.as_rdflib_graph()
requete = """
prefix : <https://urlinvente/monontologie#> 

SELECT DISTINCT ?nom
WHERE{
?s rdf:type :Animal . 
?s :nomOfficiel ?nom
}
"""
res = graph.query(requete)
for i in res:
    print(str(i[0]))

Lion
Brebis


In [27]:
if Nothing in nonsense.equivalent_to:
    print("nonsense is inconsistent!")
if Nothing in Animal.equivalent_to:
    print("Animal is inconsistent!")

nonsense is inconsistent!
